In [1]:
import os
import tensorflow_io as tfio

In [2]:
%env TFIO_GOST_DATABASE_NAME=gost
%env TFIO_GOST_DATABASE_HOST=127.0.0.1:5050
%env TFIO_GOST_DATABASE_PORT=5432
%env TFIO_GOST_DATABASE_USER=postgres
%env TFIO_GOST_DATABASE_PASS=postgres

env: TFIO_GOST_DATABASE_NAME=gost
env: TFIO_GOST_DATABASE_HOST=127.0.0.1:5050
env: TFIO_GOST_DATABASE_PORT=5432
env: TFIO_GOST_DATABASE_USER=postgres
env: TFIO_GOST_DATABASE_PASS=postgres


In [3]:
endpoint="postgresql://{}:{}@{}?port={}&dbname={}".format(
    os.environ['TFIO_GOST_DATABASE_USER'],
    os.environ['TFIO_GOST_DATABASE_PASS'],
    os.environ['TFIO_GOST_DATABASE_HOST'],
    os.environ['TFIO_GOST_DATABASE_PORT'],
    os.environ['TFIO_GOST_DATABASE_NAME'],
)

In [4]:
class Config():
    def __init__(self):
        table_names = {
            'strain1' : 'sensor_00158d00000e0ee5',
            'acc1' : 'sensor_00158d00000e0fe9',
            'acc2' : 'sensor_00158d00000e054c',
            'incl' : 'sensor_00158d00000e1024',
            'temp' : 'sensor_00158d00000e047b',
            'strain2': 'sensor_000000008bff4366'
        }
        self.table_names = table_names
        
        self.column_names = {
            table_names['strain1'] : ['id','ts','ch_mv0','ch_mv1','ch_mv2','ch_mv3'],
            table_names['acc1'] : ['id','ts','ch_x','ch_y','ch_z'],
            table_names['acc2'] : ['id','ts','ch_x','ch_y','ch_z'],
            table_names['incl'] : ['id','ts','ch_x','ch_y'],
            table_names['temp'] : ['id','ts','ch_temperature'],
            table_names['strain2'] : ['id','ts','ch_mv0','ch_mv0_379']
        }

        self.schema = 'v1'        

In [5]:
class Settings():
    def __init__(self):
        self.sensors = ['acc1']
        self.n_samples = 10

In [6]:
config = Config()
settings = Settings()

In [7]:
def generate_select():
    select_command = ''
    for sensor in settings.sensors:
        table = config.table_names[sensor]
        for column in config.column_names[table]:
            select_command+=table+'.'+column+' ,' # schema? alias?
    return select_command[:-1]

def generate_query():
    select_command = f"SELECT {generate_select()}"
    from_command =  f"FROM {config.schema}.{(', '+config.schema+'.').join([config.table_names[sensor] for sensor in settings.sensors])} "
    where_clause = f"WHERE {config.schema}.{config.table_names[settings.sensors[0]]}.id < {settings.n_samples}"
    query = select_command + from_command + where_clause
    return query



In [10]:
query = 'SELECT strain.ch_mv0, strain.id FROM v1.sensor_000000008bff43b6 strain WHERE strain.id < 10'

In [20]:
query = 'SELECT sensor_00158d00000e0fe9.id ,sensor_00158d00000e0fe9.ch_x FROM v1.sensor_00158d00000e0fe9 WHERE v1.sensor_00158d00000e0fe9.id < 10'

In [8]:
generate_query()

'SELECT sensor_00158d00000e0fe9.id ,sensor_00158d00000e0fe9.ts ,sensor_00158d00000e0fe9.ch_x ,sensor_00158d00000e0fe9.ch_y ,sensor_00158d00000e0fe9.ch_z FROM v1.sensor_00158d00000e0fe9 WHERE v1.sensor_00158d00000e0fe9.id < 10'

In [21]:
dataset = tfio.experimental.IODataset.from_sql(
    query = query,
    endpoint = endpoint
)

print(dataset.element_spec)

InvalidArgumentError: OID of data type 1700 is not supported [Op:IO>SqlIterableInit]

In [12]:
for element in dataset:
    print(element)

{'ch_mv0': <tf.Tensor: shape=(), dtype=float32, numpy=8.15625>, 'id': <tf.Tensor: shape=(), dtype=int64, numpy=1>}
{'ch_mv0': <tf.Tensor: shape=(), dtype=float32, numpy=8.24219>, 'id': <tf.Tensor: shape=(), dtype=int64, numpy=2>}
{'ch_mv0': <tf.Tensor: shape=(), dtype=float32, numpy=8.30469>, 'id': <tf.Tensor: shape=(), dtype=int64, numpy=3>}
{'ch_mv0': <tf.Tensor: shape=(), dtype=float32, numpy=8.25>, 'id': <tf.Tensor: shape=(), dtype=int64, numpy=4>}
{'ch_mv0': <tf.Tensor: shape=(), dtype=float32, numpy=8.125>, 'id': <tf.Tensor: shape=(), dtype=int64, numpy=5>}
{'ch_mv0': <tf.Tensor: shape=(), dtype=float32, numpy=8.14844>, 'id': <tf.Tensor: shape=(), dtype=int64, numpy=6>}
{'ch_mv0': <tf.Tensor: shape=(), dtype=float32, numpy=8.34375>, 'id': <tf.Tensor: shape=(), dtype=int64, numpy=7>}
{'ch_mv0': <tf.Tensor: shape=(), dtype=float32, numpy=8.16406>, 'id': <tf.Tensor: shape=(), dtype=int64, numpy=8>}
{'ch_mv0': <tf.Tensor: shape=(), dtype=float32, numpy=8.30469>, 'id': <tf.Tensor: sha